## Coursera Capstone Project

In [114]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

# Matplotlib and associated plotting modules
# import matplotlib.cm as cm
# import matplotlib.colors as colors
import matplotlib.pyplot as plt 

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import wget
import requests
import folium # map rendering library
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

latitude=38.893662
longitude=-77.023592
print('Libraries imported.')

Libraries imported.


## 1. Download the zip code list of Washington DC and load in a dataframe

In [4]:
# Download the zip code list of Washington DC using opendatasoft API into a file
url="https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=Washington+DC&rows=300&refine.state=DC"
response = requests.get(url)
jsonDataStr = json.dumps(response.json())

# with open('washington_dc_zip.json', 'w') as json_file:
#   json.dump(jsonDataStr, json_file)
# open('washington_dc_zip.json', 'w').write(jsonDataStr)

In [113]:
# Load zip code JSON to Dataframe
with open("washington_dc_zip.json") as f:
  jsonData = json.load(f)

zipcodesDf = pd.json_normalize(jsonData["records"])
zipcodesDf = zipcodesDf.drop("datasetid", axis=1).drop("recordid", axis=1).drop("record_timestamp", axis=1)
zipcodesDf = zipcodesDf.drop("fields.geopoint", axis=1).drop("geometry.coordinates", axis=1).drop("fields.state", axis=1).drop("fields.dst", axis=1)
zipcodesDf = zipcodesDf.drop("fields.timezone", axis=1).drop("geometry.type", axis=1)
zipcodesDf.rename(columns={"fields.city": "city", "fields.zip": "zip", "fields.longitude": "long", "fields.latitude": "lat"}, inplace=True)
# zipcodesDf.info()
zipcodesDf.head()

,city,zip,long,lat
0,Washington,20210,-77.014647,38.893311
1,Washington,20202,-77.014647,38.893311
2,Washington,20537,-77.025133,38.894097
3,Washington,20068,-77.014647,38.893311
4,Washington,20076,-77.014647,38.893311


In [3]:
# If multiple zip codes have same lat, long, then remove the duplicates and keep only the last one
zipcodesDf.drop_duplicates(subset=["long", "lat"], keep="last", inplace=True)

In [4]:
print("Shape of the data is {}".format(zipcodesDf.shape))

Shape of the data is (98, 4)


## Using Foursquare API, get the recommended venue list for each zip code / lat long from

In [8]:
## hidden
CLIENT_ID = "YIOJTSFKMWE35RORHMBN0AMLFEEGDU4C4KGH3M3VHDRTL43L"
CLIENT_SECRET = "FVH0DHE5M11DHG2TV2XMVGOL2OKAOMXW1DEATV33RAZKVSLH"
VERSION = '20180605' # Foursquare API version
LIMIT=50

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YIOJTSFKMWE35RORHMBN0AMLFEEGDU4C4KGH3M3VHDRTL43L
CLIENT_SECRET:FVH0DHE5M11DHG2TV2XMVGOL2OKAOMXW1DEATV33RAZKVSLH


In [76]:
# FOUR_SQUARE_EXPLORE_URL = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v=20180602&limit={}".format(CLIENT_ID, CLIENT_SECRET, LIMIT)

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zip', 
                  'zip latitude', 
                  'zip longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
# Read from Foursquare and write this data to CSV for later reload so that we don't run into quota limits of FourSquare APIs
# venuesDf = getNearbyVenues(zipcodesDf["zip"], zipcodesDf["lat"], zipcodesDf["long"])
# venuesDf.to_csv("foursquare_venues.csv", index=False)

venuesDf = pd.read_csv("foursquare_venues.csv")
venuesDf.head()

,zip,zip latitude,zip longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,20065,38.883412,-77.028198,ARTECHOUSE,38.884299,-77.029266,Art Gallery
1,20065,38.883412,-77.028198,"Mandarin Oriental, Washington DC",38.883659,-77.030319,Hotel
2,20065,38.883412,-77.028198,International Spy Museum,38.883895,-77.025539,Museum
3,20065,38.883412,-77.028198,Maine Avenue Fish Market,38.881145,-77.028118,Fish Market
4,20065,38.883412,-77.028198,Falafel Inc,38.881443,-77.027500,Falafel Restaurant


In [26]:
venuesDf.groupby('zip').count().head()

,zip latitude,zip longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
zip,,,,,,
20001,14,14,14,14,14,14
20002,50,50,50,50,50,50
20003,50,50,50,50,50,50
20004,50,50,50,50,50,50
20005,50,50,50,50,50,50


In [7]:
print('There are {} unique categories.'.format(len(venuesDf['Venue Category'].unique())))
print('Shape of venues is {} '.format(venuesDf.shape))
venuesDf.head()

There are 278 unique categories.
Shape of venues is (3776, 7) 


,zip,zip latitude,zip longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,20065,38.883412,-77.028198,ARTECHOUSE,38.884299,-77.029266,Art Gallery
1,20065,38.883412,-77.028198,"Mandarin Oriental, Washington DC",38.883659,-77.030319,Hotel
2,20065,38.883412,-77.028198,International Spy Museum,38.883895,-77.025539,Museum
3,20065,38.883412,-77.028198,Maine Avenue Fish Market,38.881145,-77.028118,Fish Market
4,20065,38.883412,-77.028198,Falafel Inc,38.881443,-77.027500,Falafel Restaurant


## Analyze each zip code

In [14]:
# one hot encoding
dc_onehot = pd.get_dummies(venuesDf[['Venue Category']], prefix="", prefix_sep="")

# add zip column back to dataframe
# dc_onehot['zip'] = venuesDf['zip'] 
dc_onehot.insert(0, "zip", venuesDf['zip'])
dc_onehot.insert(1, "lat", venuesDf['zip latitude'])
dc_onehot.insert(2, "long", venuesDf['zip longitude'])

# # move zip column to the first column
# fixed_columns = [dc_onehot.columns[-1]] + list(dc_onehot.columns[:-1])
# dc_onehot = dc_onehot[fixed_columns]

dc_onehot.head()

,zip,lat,long,ATM,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Check Cashing Service,Chinese Restaurant,Christmas Market,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Courthouse,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wi

In [17]:
print('Shape is {} '.format(dc_onehot.shape))

Shape is (3776, 281) 


### Next, let's group rows by zip code and by taking the mean of the frequency of occurrence of each category

In [15]:
dc_grouped = dc_onehot.groupby('zip').mean().reset_index()
dc_grouped.head()

,zip,lat,long,ATM,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Check Cashing Service,Chinese Restaurant,Christmas Market,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Courthouse,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wi

In [16]:
print('Shape is {} '.format(dc_grouped.shape))

Shape is (98, 281) 


In [115]:
# create map
map_city = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, zipcode in zip(dc_grouped['lat'], dc_grouped['long'], dc_grouped['zip']):
    label = 'Zip:{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  
    
map_city

### Create a new dataframe with only "Restaurant" columns

In [26]:
# Create a new dataframe with only "* Restaurant *" columns
cols = [col for col in dc_grouped.columns]
restaurantCategories = [col for col in dc_grouped.columns if col.find("Restaurant") >= 0]
# cols.insert(0, "zip")
print('Total number of restaurant categories are {} '.format(len(restaurantCategories)))

Total number of restaurant categories are 52 


In [90]:
restaurantsDf = dc_grouped[restaurantCategories]
# restaurantsDf.loc[0:, "Total"] = restaurantsDf.sum(axis=1)
restaurantsDf.insert(0, "Score", restaurantsDf.sum(axis=1) )
restaurantsDf.insert(0, "zip", dc_grouped["zip"] )
restaurantsDf.insert(1, "lat", dc_grouped["lat"] )
restaurantsDf.insert(2, "long", dc_grouped["long"] )
# restaurantsDf.iloc[0]
print('Shape of restaurants dataframe with scores {} '.format(restaurantsDf.shape))
restaurantsDf.head()

Shape of restaurants dataframe with scores (98, 56) 


,zip,lat,long,Score,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Seafood Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,20001,38.907711,-77.01732,0.142857,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.071429,0.00,0.00,0.00,0.0,0.071429,0.0,0.00,0.0,0.0
1,20002,38.901811,-76.99097,0.240000,0.0,0.02,0.00,0.04,0.00,0.00,0.02,0.02,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.02,0.00,0.02,0.02,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.020000,0.00,0.02,0.02,0.0,0.020000,0.0,0.00,0.0,0.0
2,20003,38.881762,-76.99447,0.340000,0.0,0.04,0.00,0.02,0.02,0.00,0.00,0.00,0.0,0.0,0.02,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.04,0.0,0.00,0.00,0.0,0.0,0.02,0.00,0.02,0.00,0.0,0.0,0.0,0.0,0.04,0.0,0.02,0.0,0.000000,0.02,0.04,0.00,0.0,0.000000,0.0,0.00,0.0,0.0
3,20004,38.895268,-77.02760,0.160000,0.0,0.04,0.00,0.00,0.00,0.02,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.02,0.00,0.0,0.0,0.0,0.0,0.02,0.0,0.02,0.0,0.020000,0.00,0.02,0.00,0.0,0.000000,0.0,0.00,0.0,0.0
4,20005,38.904461,-77.03088,0.280000,0.0,0.04,0.02,0.00,0.02,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.02,0.0,0.02,0.02,0.0,0.0,0.02,0.02,0.04,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.00,0.0,0.000000,0.0,0.04,0.0,0.0


In [91]:
# Sort zip codes with total restaurant score in descending order
sortedZipcodes = restaurantsDf.sort_values('Score', ascending=False).reset_index(drop=True).head(5)
sortedZipcodes.reset_index(drop=True)
sortedZipcodes.head()

,zip,lat,long,Score,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Seafood Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,20548,38.898111,-77.017698,0.440000,0.0,0.060000,0.0,0.020000,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.040000,0.00,0.000000,0.040000,0.000000,0.0,0.0,0.040000,0.04,0.00,0.000000,0.0,0.0,0.020000,0.020000,0.040000,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.02,0.00,0.020000,0.020000,0.0,0.0
1,20586,38.902211,-77.047399,0.440000,0.0,0.100000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.04,0.00,0.0,0.0,0.040000,0.02,0.020000,0.020000,0.000000,0.0,0.0,0.020000,0.00,0.02,0.020000,0.0,0.0,0.040000,0.000000,0.000000,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.00,0.02,0.020000,0.000000,0.0,0.0
2,20055,38.901633,-77.020959,0.440000,0.0,0.080000,0.0,0.020000,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.020000,0.00,0.040000,0.040000,0.020000,0.0,0.0,0.060000,0.00,0.00,0.020000,0.0,0.0,0.000000,0.020000,0.020000,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.0,0.02,0.00,0.000000,0.040000,0.0,0.0
3,20442,38.895977,-77.017668,0.420000,0.0,0.060000,0.0,0.020000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.02,0.0,0.0,0.020000,0.00,0.020000,0.040000,0.000000,0.0,0.0,0.020000,0.04,0.00,0.040000,0.0,0.0,0.020000,0.000000,0.040000,0.0,0.0,0.04,0.0,0.0,0.0,0.02,0.0,0.02,0.00,0.000000,0.000000,0.0,0.0
4,20536,38.901211,-77.016948,0.408163,0.0,0.081633,0.0,0.020408,0.0,0.020408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.020408,0.00,0.040816,0.040816,0.020408,0.0,0.0,0.020408,0.00,0.00,0.020408,0.0,0.0,0.020408,0.020408,0.040816,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.020408,0.020408,0.0,0.0


### Show the PastaPlace company recommendations on map

In [109]:
sortedZipcodes["zip"].tolist()

[20548, 20586, 20055, 20442, 20536]

In [110]:
# create map
latitude=38.893662
longitude=-77.023592

map_city = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, zipcode, score in zip(sortedZipcodes['lat'], sortedZipcodes['long'], sortedZipcodes['zip'], sortedZipcodes.index.values):
    label = 'Zip:{}, Rank:{}'.format(zipcode, score+1)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  
    
map_city